In [1]:
import json


In [2]:


classes = [
    {
        "Order": {
            "orderId": "String",
            "version": "Long",
            "orderType": "OrderType",
            "orderSide": "OrderSide",
            "status": "Status",
            "allocations": "List<Allocation>"
        }
    },
    {
        "OrderType": [
            "MarketOrderType",
            "LimitOrderType"
        ]
    },
    {
        "MarketOrderType": ""
    },
    {
        "LimitOrderType": {
            "price": "Double"
        }
    },
    {
        "OrderSide": [
            "Buy",
            "Sell"
        ]
    },
    {
        "Status": [
            "New",
            "Verifying",
            "Pending",
            "Working",
            "PartiallyFilled",
            "Filled",
            "Cancelled"
        ]
    },
    {
        "Allocation": [
            "LongAllocation",
            "EmptyAllocation"
        ]
    },
    {
        "LongAllocation": {
            "clientName": "String"
        }
    },
    {
        "EmptyAllocation": ""
    }
]
statements = [
    "O",
    "Order.",
    "Order.orderType.",
    "Order.order",
    "OrderType.LimitOrderType.pri",
    "Order.orderType.LimitOrderType.price",
    "Order.allocations.",
    "Status.P",
    "MarketOrderType.",
    ".Hello.",
    "HELL"
]

In [3]:
parse_data = json.loads(json.dumps(classes))
for i in classes:
    print(i)
    

{'Order': {'orderId': 'String', 'version': 'Long', 'orderType': 'OrderType', 'orderSide': 'OrderSide', 'status': 'Status', 'allocations': 'List<Allocation>'}}
{'OrderType': ['MarketOrderType', 'LimitOrderType']}
{'MarketOrderType': ''}
{'LimitOrderType': {'price': 'Double'}}
{'OrderSide': ['Buy', 'Sell']}
{'Status': ['New', 'Verifying', 'Pending', 'Working', 'PartiallyFilled', 'Filled', 'Cancelled']}
{'Allocation': ['LongAllocation', 'EmptyAllocation']}
{'LongAllocation': {'clientName': 'String'}}
{'EmptyAllocation': ''}


In [4]:
def parse_classes(classes):
    store_objects = {}
    store_variables = {}
    for i in classes:
        for key, value in i.items():
            if isinstance(value, dict):
                store_objects[key] = list(value.keys())

                for sub_key, sub_value in value.items():
                    store_variables[f'{key}.{sub_key}'] = sub_value

            elif isinstance(value, list):
                store_objects[key] = value
                for sub_key in value:
                    store_variables[f'{key}.{sub_key}'] = ""
            
            elif isinstance(value, str):
                store_objects[key] = [value]
                store_variables[f'{key}.{value}'] = ""

    return store_objects, store_variables

store_objects, store_variables = parse_classes(classes)
print(store_objects)
print(store_variables)


{'Order': ['orderId', 'version', 'orderType', 'orderSide', 'status', 'allocations'], 'OrderType': ['MarketOrderType', 'LimitOrderType'], 'MarketOrderType': [''], 'LimitOrderType': ['price'], 'OrderSide': ['Buy', 'Sell'], 'Status': ['New', 'Verifying', 'Pending', 'Working', 'PartiallyFilled', 'Filled', 'Cancelled'], 'Allocation': ['LongAllocation', 'EmptyAllocation'], 'LongAllocation': ['clientName'], 'EmptyAllocation': ['']}
{'Order.orderId': 'String', 'Order.version': 'Long', 'Order.orderType': 'OrderType', 'Order.orderSide': 'OrderSide', 'Order.status': 'Status', 'Order.allocations': 'List<Allocation>', 'OrderType.MarketOrderType': '', 'OrderType.LimitOrderType': '', 'MarketOrderType.': '', 'LimitOrderType.price': 'Double', 'OrderSide.Buy': '', 'OrderSide.Sell': '', 'Status.New': '', 'Status.Verifying': '', 'Status.Pending': '', 'Status.Working': '', 'Status.PartiallyFilled': '', 'Status.Filled': '', 'Status.Cancelled': '', 'Allocation.LongAllocation': '', 'Allocation.EmptyAllocation

In [5]:
# UNDYNAMIC BRUTE FORCE-ISH SOLUTION JUST FOR FINDING A PATTERN

def get_next_words(state, store_objects, store_variables):
    #Check for test cases: If Objects Don't exist in the first place:
    print(state)
    #Base case(s):
    if state[0] not in store_objects:
        if len(state) == 1:
            a = store_objects.keys()
            return [x for x in a if x.startswith(state[0])]
        return [""]
    result = [""]
    flag = True

    #Need to find patterns first.
    #First check for the condition in which there only exists 2 words (E.g. Order.) and cases that handles these():
    if len(state) == 2:
        if state[1] == '':
            return store_objects[state[0]]
        #For cases like ['Status', 'P'], no spaces in between
        #Can handle edge cases like double dots.
        return [x for x in store_objects[state[0]] if x.startswith(state[1])]
 
    
    #Check for condition in which there exists 3 words (E.g. Order.allocations.):
    if len(state) == 3:
        #Order allocations " "
        combined = state[0] + "." + state[1]
        flag =  0
        if combined in store_variables:
            print("hi")
            combined = store_variables[combined]
            flag = 1
        if flag:
            print(state)
            if state[-1] == '':
                if combined not in store_objects:
                    return [""]
                return store_objects[combined]

    if len(state) == 4:
        combined = state[0] + "." + state[1]
        flag =  0
        if combined in store_variables:
            print("hi")
            combined = store_variables[combined]
            flag = 1
        print(combined)
        if flag:
            combined = combined + "." + state[2]
            if combined in store_variables:
                combined = store_variables[combined]
            else:
                flag = 0
            
            if flag:
                if state[-1] == '':
                    if combined not in store_objects:
                        return [""]
                    return store_objects[combined]
                else:
                    print("gii")
                    return [x for x in store_objects[combined] if x.startswith(state[4])]

    return result

In [6]:
def get_next_words(state, store_objects, store_variables):
    # Check if state[0] exists in store_objects
    if state[0] not in store_objects:
        if len(state) == 1:
            # If only the object name is given, list out all objects that start with it
            return [x for x in store_objects.keys() if x.startswith(state[0])]
        return [""]
    
    # Initialize combined and current_type with the first object in state
    combined = current_type = state[0]
    
    # Loop through all the words in the state except the last one
    for i in range(1, len(state) - 1):
        # Update combined to build the key for store_variables
        combined += "." + state[i]
        # If the combined key exists in store_variables, update current_type
        if combined in store_variables:
            if store_variables[combined] == "":
                print()
            current_type = store_variables[combined]
    
    # If the current_type is not in store_objects, return [""]
    if current_type not in store_objects:
        return [""]
    
    # For the last word in state, if it's an empty string, return all the variables/methods under current_type
    # Else, return all the variables/methods that start with it
    if state[-1] == '':
        return store_objects[current_type]
    else:
        return [x for x in store_objects[current_type] if x.startswith(state[-1])]

# Main program
ret = {}
for statement in statements:
    state = statement.split(".")
    list_params = get_next_words(state, store_objects, store_variables)
    list_params.sort()
    if len(list_params) >= 5:
        ret[statement] = list_params[:5]
    else:
        ret[statement] = list_params
print(ret)


{'O': ['Order', 'OrderSide', 'OrderType'], 'Order.': ['allocations', 'orderId', 'orderSide', 'orderType', 'status'], 'Order.orderType.': ['LimitOrderType', 'MarketOrderType'], 'Order.order': ['orderId', 'orderSide', 'orderType'], 'OrderType.LimitOrderType.pri': [''], 'Order.orderType.LimitOrderType.price': [], 'Order.allocations.': [''], 'Status.P': ['PartiallyFilled', 'Pending'], 'MarketOrderType.': [''], '.Hello.': [''], 'HELL': []}


In [7]:
ret = {}
for statement in statements:
    state = statement.split(".")
    list_params = get_next_words(state, store_objects, store_variables)
    list_params.sort()
    if len(list_params) >= 5:
        ret[statement] = list_params[:5]
    else:
        ret[statement] = list_params
print(ret)


{'O': ['Order', 'OrderSide', 'OrderType'], 'Order.': ['allocations', 'orderId', 'orderSide', 'orderType', 'status'], 'Order.orderType.': ['LimitOrderType', 'MarketOrderType'], 'Order.order': ['orderId', 'orderSide', 'orderType'], 'OrderType.LimitOrderType.pri': [''], 'Order.orderType.LimitOrderType.price': [], 'Order.allocations.': [''], 'Status.P': ['PartiallyFilled', 'Pending'], 'MarketOrderType.': [''], '.Hello.': [''], 'HELL': []}
